In [133]:
from funcx.sdk.client import FuncXClient
import time


fxc = FuncXClient()
fxc.throttling_enabled = False
print(fxc.base_url, fxc.throttling_enabled)

https://funcx.org/api/v1 False


In [143]:
endpoint_uuid = '709118de-1103-463f-8425-281eb93b55ff' #Theta covid19
#endpoint_uuid = '67e95158-8bda-4b1f-a0ef-31a1626eba00' #Ryan ep

In [135]:
def generate_batch(filename, start=0, batchsize=10, max_batches=10):

    counter = 0
    if max_batches == 0:
        max_batches = 999999999

    x = 'Hello'
    batch_index = []
    with open(filename) as current:
        batch_index.append(current.tell())
        counter += 1

        while x and counter < max_batches:
            counter += 1
            for i in range(batchsize):
                x = current.readline()

            batch_index.append(current.tell())
        return batch_index

In [136]:
func_uuid1 = fxc.register_function(generate_batch,
                                   description="A funcx function for generating batch")

print(func_uuid1)

75deaa6d-5555-4fd9-b7c4-de6373222861


In [144]:
res1 = fxc.run(filename='//projects/CSC249ADCD01/zz/ScreenPilot/ena+db.can',
               endpoint_id=endpoint_uuid,
               function_id=func_uuid1)

print(res1)

1cf19df8-cce0-4ca6-b218-7a005748195e


In [146]:
batch_index = fxc.get_result(res1)
print(batch_index)

[0, 2321, 3153, 3480, 4006, 4429, 4950, 5357, 5994, 6402]


In [101]:
def funcx_runner(index, filename, batchsize, index_start, index_end, workdir=None, out_file=None):
    from candle_apps.candle_node_local import funcx_node_local
    return funcx_node_local(filename, index, batchsize, index_start, index_end,
                            workdir=workdir, out_file=out_file)

In [102]:
func_uuid2 = fxc.register_function(funcx_runner,
                                   description="A funcx function for covid19")

print(func_uuid2)

9734b381-7716-4a43-8ba4-3852703c8e3c


### We can use funcx's batch submit?

In [130]:
task_ids = []
for idx in batch_index:
    res = fxc.run(filename='/home/zzli/candle/ScreenPilot/ena+db.can',
                  index=idx,
                  batchsize=10,
                  index_start=0,
                  index_end=1000,
                  workdir='/tmp/zzli/',
                  out_file='/home/zzli/candle/ScreenPilot/data-{}'.format(idx),
                  endpoint_id=endpoint_uuid,
                  function_id=func_uuid2)
    task_ids.append(res)

print(task_ids)

['3c852742-7c62-4fc2-b242-07448bef5d8e', '2ac60e2a-9e3a-4dde-818e-5718f28ad4ce', 'ed9ece9d-4841-40e6-a74a-d5ebfb466e2d', '267fbdf1-6bea-41e2-8f21-7ef8f014114f', '8d220133-99ef-467a-b4a5-e584de50d7fe', 'fac6a7d2-b2e3-4c3c-a1a9-260d49ca7f63', '3cb2fcac-cf2b-4fad-9407-cd9e00f15d4f', '23445866-098a-447e-8eed-632d935ddf93', '309d80b1-2113-47ba-8ef7-b7a061950330', 'e2cc5d3c-7c2e-4fce-bd3a-3b0ee18fbe16']


In [132]:
x = fxc.get_batch_status(task_ids)
print(x)
try:
    print(x[res[0]]['exception'].reraise())
except:
    pass

{'3c852742-7c62-4fc2-b242-07448bef5d8e': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-0'}, '2ac60e2a-9e3a-4dde-818e-5718f28ad4ce': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-2321'}, 'ed9ece9d-4841-40e6-a74a-d5ebfb466e2d': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-3153'}, '267fbdf1-6bea-41e2-8f21-7ef8f014114f': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-3480'}, '8d220133-99ef-467a-b4a5-e584de50d7fe': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-4006'}, 'fac6a7d2-b2e3-4c3c-a1a9-260d49ca7f63': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-4429'}, '3cb2fcac-cf2b-4fad-9407-cd9e00f15d4f': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-4950'}, '23445866-098a-447e-8eed-632d935ddf93': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-5357'}, '309d80b1-2113-47ba-8ef7-b7a061950330': {'pending': 'False', 'result': '/home/zzli